In [ ]:
from scapy.all import sniff, conf
import csv
import threading
from datetime import datetime
import queue
import time

'''
conda activate netsec_env
spyder
'''

# scapy layer-3 socket config for sniffing
conf.l3socket = conf.L3socket

# csv generation
CSV_FILE = 'netlog.csv'

# packet queue to push to netlog
packet_queue = queue.Queue()

# generate header
def init_csv():
    try:
        with open(CSV_FILE, mode='x', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([
                "timestamp", "src_ip", "dst_ip", "protocol", "length",
                "src_port", "dst_port", "flags", "ttl", "payload_size", "protocol_name"
            ])
    except FileExistsError:
        pass

# push packet data from the queue into the netlog
def write_to_csv():
    while True:
        packet_data = packet_queue.get()
        if packet_data is None:  # exit condition for threaded comp
            break
        try:
            with open(CSV_FILE, mode='a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(packet_data)
        except Exception as e:
            print(f"Error writing packet to CSV: {e}")
        packet_queue.task_done()

# callback for sniffing packets
def packet_callback(packet):
    try:
        # fetch timestamp
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        if packet.haslayer('IP'):
            src_ip = packet['IP'].src
            dst_ip = packet['IP'].dst
            proto = packet.proto
            length = len(packet)
            ttl = packet['IP'].ttl

            # assign protocol name based on packet protocol
            if proto == 1:
                protocol_name = 'ICMP'
            elif proto == 6:
                protocol_name = 'TCP'
            elif proto == 17:
                protocol_name = 'UDP'
            else:
                protocol_name = 'Other'

            # init for default TCP/UDP data
            src_port = dst_port = flags = payload_size = None

            # if TCP packet, extract port numbers and flags
            if packet.haslayer('TCP'):
                src_port = packet['TCP'].sport
                dst_port = packet['TCP'].dport
                flags = packet['TCP'].flags
                payload_size = len(packet['TCP'].payload)

            # if UDP packet, extract port numbers (no flags)
            elif packet.haslayer('UDP'):
                src_port = packet['UDP'].sport
                dst_port = packet['UDP'].dport
                flags = None
                payload_size = len(packet['UDP'].payload)

            # enqueue packets for logging
            packet_data = [
                timestamp, src_ip, dst_ip, proto, length,
                src_port, dst_port, flags, ttl, payload_size, protocol_name
            ]
            packet_queue.put(packet_data)
    except Exception as e:
        print(f"Error processing packet: {e}")

# spin the packet-write thread
def start_sniffing():
    print("Sniffing started... Press Ctrl+C to stop.")
    sniff(prn=packet_callback, store=False)

# init the netlog and start writer thread
init_csv()

# spin the background thread for writing to log
write_thread = threading.Thread(target=write_to_csv, daemon=True)
write_thread.start()

# start sniffing
try:
    start_sniff_
